# Feature Engineering
We are using `Pandas` for feature engineering. Our goal in this file is to process following features:
1. Loan Amount
2. Age
3. Square feet
4. Property Type
5. Weekday
6. Year sold
7. Zillow data

In [1]:
import pandas as pd
import numpy as np
import os, sys
from pprint import pprint

In [2]:
DATA_DIR = "../data/"
RAW_DATA_FILE = DATA_DIR + "data.csv"

# Check if data dir or file exist
if not os.path.exists(DATA_DIR) \
    or not os.path.exists(RAW_DATA_FILE):
    print(
        "CANNOT find {} or {}".format(DATA_DIR, RAW_DATA_FILE),
        file=sys.stderr
    )
    sys.exit()

# Load csv data
raw_data = pd.read_csv(RAW_DATA_FILE)

# Columns that we think useful
COLS = [
            "property_id",
            "transaction_amount",
            "loan_amount",
            "transaction_date",
            "property_type",
            "year_built",
            "sqft",
            "property_address"
        ]

data = raw_data[COLS]

In [3]:
data = data.loc[ data['transaction_amount'] < 13000000 ]
data = data.loc[ (data['year_built'] != 0) & (data['sqft'] != 0) ]
data.head(10)

,property_id,transaction_amount,loan_amount,transaction_date,property_type,year_built,sqft,property_address
0,27885942,670000,0.0,2012-02-27,RCON,1993,2682,NaN
1,26400001,260000,0.0,2012-04-12,RSFR,1984,1452,NaN
2,31540676,190000,0.0,2012-04-18,RSFR,1903,1891,NaN
3,23913396,136500,0.0,2012-04-27,RSFR,1994,1147,NaN
4,28597691,21500,0.0,2012-05-09,VRES,1977,1462,NaN
6,25794203,220000,0.0,2012-06-15,RSFR,1977,1453,725 15TH AVE ESCONDIDO CA 920255539
7,109702967,789700,0.0,2012-06-21,RSFR,2005,3551,NaN
8,26552359,0,160000.0,2012-09-06,RSFR,1955,1236,8423 VIA LADERA RANCHO CUCAMONGA CA 917301047
9,29260978,106500,0.0,2012-09-13,RSFR,1963,1613,NaN
10,31084594,123500,0.0,2012-09-14,RSFR,1984,1962,NaN


In [4]:
# Create index for appending columns by `join`
idx = data.index.tolist()


__Target features__:
1. Loan Amount
2. Age
3. Square feet
4. Property Type
5. Weekday
6. Year sold
7. Zillow data

## Age & Year built

In [7]:
from datetime import datetime
# Column of transaction date
transyrs = []  # Transaction date, years
transages = []  # Property age when transaction happens
transwkds = []  # Transaction date, weekdays
nz_yrblt_cnt, nz_age_sum = 0, 0  # Non-zero year built count and Non-zero year built sum of age

for index, row in data.iterrows():
    transdate, yrblt = row['transaction_date'], row['year_built']
    transdate_obj  = datetime.strptime(transdate, "%Y-%m-%d")  
        # All date are valid, no need for exception handling
    
    transyrs.append(transdate_obj.year)
    transwkds.append(transdate_obj.isoweekday())
    
    if yrblt != 0:
        age = transdate_obj.year - yrblt
        nz_yrblt_cnt += 1
        nz_age_sum += age
        transages.append(age)
    else:
        # If year built is 0, append -1. Change each -1 later
        transages.append(-1)

# Compute average age
avg_age = nz_age_sum// nz_yrblt_cnt
    
for i in range(len(transages)):
    if transages[i] < 0:
        transages[i] = avg_age

In [8]:
idx = data.index.tolist()

## Appending the columns the attributes by join (ages, transaction_year, and weekdays)

In [9]:
newdata = data.copy()
print(newdata.shape)

(149199, 8)


In [10]:
# Join age
data_age = pd.DataFrame(index=idx, data=transages, columns=['age'], dtype=np.int64)
newdata = newdata.join(data_age)

In [11]:
# Join weekday
data_wkd = pd.DataFrame(index=idx, data=transwkds, columns=['weekdays'], dtype=np.int64)
newdata = newdata.join(data_wkd)

In [12]:
# Join transaction_years
data_yrs = pd.DataFrame(index=idx, data=transyrs, columns=['transaction_year'], dtype=np.int64)
newdata = newdata.join(data_yrs)

In [14]:
newdata.shape # Should be same length, 3 more columns

(149199, 11)

## Address

In [15]:
data_addr = data['property_address']
print("How many address are NaN? ", data_addr.isnull().sum())

How many address are NaN?  58398


In [18]:
print(newdata.shape)

(149199, 11)


In [19]:
panull = data[data['property_address'].isnull()]

In [20]:
count_yb = panull['year_built'].value_counts()
count_yb.sort_index()

1801       7
1808       1
1850       2
1860       1
1870       5
1875       1
1878       1
1879       1
1880       7
1881       1
1885       5
1886       2
1888       4
1889       3
1890      37
1891       2
1892       5
1893       3
1894       3
1895      24
1896       6
1897       6
1898      10
1899       2
1900     120
1901      23
1902      24
1903      37
1904      34
1905      69
        ... 
1988     940
1989    1061
1990    1004
1991     629
1992     480
1993     419
1994     403
1995     362
1996     324
1997     313
1998     354
1999     421
2000     501
2001     542
2002     584
2003     732
2004     941
2005     970
2006     943
2007     544
2008     268
2009     126
2010     107
2011      47
2012     114
2013     476
2014     639
2015     655
2016     440
2017      65
Name: year_built, Length: 142, dtype: int64

In [21]:
count_pt = panull['property_type'].value_counts()
count_pt.sort_index()

RAPT      395
RCON     6478
RCOO        5
RDUP     1178
RMFD      449
RMOB      913
RMSC      529
RQUA      330
RSFR    47535
RTIM        1
RTRI      284
VRES      301
Name: property_type, dtype: int64

For previous two steps, we are trying to find common traits that don't have an address. It seems that such trait doesn't exist.

### Integrating Zillow dataset

In [22]:
ZILLOW_DATA_FILE = DATA_DIR + "zillow.csv"
zillowdata = pd.read_csv(ZILLOW_DATA_FILE)
zillowdata = zillowdata[zillowdata['State'] == 'CA'] # Extract California indexes

In [23]:
# Extracting zipcode from the `property_address` column.
# Searching the zipcode from zillow dataset, creating new columns
zipcodes = []
pzhvis, zhvis = [], []
nan_cnt = 0
weight_sum = 0.0
zhvi_sum, pzhvi_sum = 0.0, 0.0
rankzero = 0
for index, addr in data_addr.iteritems():
    if not isinstance(addr, str):
        # zipcodes.append(-1)
        zhvis.append(-1)
        pzhvis.append(-1)
        nan_cnt += 1
    else:
        zipcode = addr.split(" ")[-1]
        if zipcode == "CA":
            zhvis.append(-1)
            pzhvis.append(-1)
            nan_cnt += 1
        else:
            zipcode = zipcode[:5]
            row = zillowdata[zillowdata['RegionName'] == int(zipcode)]
            rank = row['SizeRank']
            
            if rank.empty:
                zhvis.append(-1)
                pzhvis.append(-1)
                nan_cnt += 1
                rankzero +=1
            else:
                rank = float(rank)
                zhvi, pzhvi = float(row['Zhvi']), float(row['PeakZHVI'])
                weight_sum += 1 / rank
                zhvi_sum += zhvi / rank
                pzhvi_sum += pzhvi / rank
                zhvis.append(zhvi)
                pzhvis.append(pzhvi)         
    

In [24]:
print(len(pzhvis), len(zhvis))

149199 149199


In [25]:
# For those ZHVI and PZHVI are unavailable, fill in AVG(ZHVI) and AVG(PZHVI)
for i in range(len(pzhvis)):
    if pzhvis[i] < 0:
        pzhvis[i] = int(pzhvi_sum / weight_sum)
        zhvis[i] = int(zhvi_sum / weight_sum)

In [26]:
data_zhvi = pd.DataFrame(index=idx, columns=['zhvi'], data=zhvis)
newdata = newdata.join(data_zhvi)

data_pzhvi = pd.DataFrame(index=idx, columns=['pzhvi'], data=pzhvis)
newdata = newdata.join(data_pzhvi)

newdata.head(10)
# Seems all correct

,property_id,transaction_amount,loan_amount,transaction_date,property_type,year_built,sqft,property_address,age,weekdays,transaction_year,zhvi,pzhvi
0,27885942,670000,0.0,2012-02-27,RCON,1993,2682,NaN,19,1,2012,629346.0,656452.0
1,26400001,260000,0.0,2012-04-12,RSFR,1984,1452,NaN,28,4,2012,629346.0,656452.0
2,31540676,190000,0.0,2012-04-18,RSFR,1903,1891,NaN,109,3,2012,629346.0,656452.0
3,23913396,136500,0.0,2012-04-27,RSFR,1994,1147,NaN,18,5,2012,629346.0,656452.0
4,28597691,21500,0.0,2012-05-09,VRES,1977,1462,NaN,35,3,2012,629346.0,656452.0
6,25794203,220000,0.0,2012-06-15,RSFR,1977,1453,725 15TH AVE ESCONDIDO CA 920255539,35,5,2012,511100.0,523600.0
7,109702967,789700,0.0,2012-06-21,RSFR,2005,3551,NaN,7,4,2012,629346.0,656452.0
8,26552359,0,160000.0,2012-09-06,RSFR,1955,1236,8423 VIA LADERA RANCHO CUCAMONGA CA 917301047,57,4,2012,423400.0,447400.0
9,29260978,106500,0.0,2012-09-13,RSFR,1963,1613,NaN,49,4,2012,629346.0,656452.0
10,31084594,123500,0.0,2012-09-14,RSFR,1984,1962,NaN,28,5,2012,629346.0,656452.0


In [27]:
newdata.columns

Index(['property_id', 'transaction_amount', 'loan_amount', 'transaction_date',
       'property_type', 'year_built', 'sqft', 'property_address', 'age',
       'weekdays', 'transaction_year', 'zhvi', 'pzhvi'],
      dtype='object')

In [28]:
newdata.shape

(149199, 13)

## Forming the categorical data
Here are three attributes: property_type, weekday, and transaction_year. They are categorical features, we decide to use binary encoder to transfer them into numberical feature. 

In [29]:
import category_encoders as ce
X = pd.DataFrame(newdata, 
                 columns=[
                     'loan_amount', 
                     'property_type', 
                     'sqft', 
                     'age', 
                     'weekday',
                     'transaction_year',
                     'zhvi',
                     'pzhvi']
                )

y = pd.DataFrame(newdata,
                columns=['transaction_amount'])

In [30]:
print(X.shape)
print(y.shape)

(149199, 8)
(149199, 1)


In [31]:
enc = ce.BinaryEncoder(cols=['property_type', 'weekday', 'transaction_year']).fit(X,y)
numX  = enc.transform(X)
numX.head(30)

,property_type_0,property_type_1,property_type_2,property_type_3,transaction_year_0,transaction_year_1,transaction_year_2,loan_amount,sqft,age,zhvi,pzhvi
0,0,0,0,0,0,0,0,0.0,2682,19,629346.0,656452.0
1,0,0,0,1,0,0,0,0.0,1452,28,629346.0,656452.0
2,0,0,0,1,0,0,0,0.0,1891,109,629346.0,656452.0
3,0,0,0,1,0,0,0,0.0,1147,18,629346.0,656452.0
4,0,0,1,0,0,0,0,0.0,1462,35,629346.0,656452.0
6,0,0,0,1,0,0,0,0.0,1453,35,511100.0,523600.0
7,0,0,0,1,0,0,0,0.0,3551,7,629346.0,656452.0
8,0,0,0,1,0,0,0,160000.0,1236,57,423400.0,447400.0
9,0,0,0,1,0,0,0,0.0,1613,49,629346.0,656452.0
10,0,0,0,1,0,0,0,0.0,1962,28,629346.0,656452.0


## Store the processed dataset into disk
The processed dataset are stored into DATA_DIR/dataX.csv and DATA_DIR/dataY.csv

In [32]:
TARGET_FILE_X = DATA_DIR + "dataX.csv"
TARGET_FILE_Y = DATA_DIR + "dataY.csv"
numX.to_csv(TARGET_FILE_X)
y.to_csv(TARGET_FILE_Y)